# References
https://github.com/lukewrites/NP_chunking_with_nltk/blob/master/NP_chunking_with_the_NLTK.ipynb

In [1]:
import nltk
import re
import pprint
from nltk import Tree

from datasets.np_chunks import NPExtractor

In [2]:
np_extractor = NPExtractor()
index = 56



In [3]:
from configs.args import load_arg_parser
import os
parser = load_arg_parser()
cfg = parser.parse_args("")
cfg.data_root = "/data/aiyucui2/wider"
root = cfg.data_root

# data path
cfg.anno_path = os.path.join(root, cfg.anno_path)
cfg.img_dir = os.path.join(root, cfg.img_dir)
cfg.val_anno_path = os.path.join(root, cfg.val_anno_path)
cfg.val_img_dir = os.path.join(root, cfg.val_img_dir)
cfg.gt_file_fn = os.path.join(root, cfg.gt_file_fn)

# meta data path
cfg.cheap_candidate_fn = os.path.join(root, cfg.cheap_candidate_fn)
cfg.vocab_path = os.path.join(root, cfg.vocab_path)

# sys path
cfg.model_path = os.path.join(root, cfg.model_path)
cfg.output_path = os.path.join(root, cfg.output_path)
cfg.debug = False
cfg.embed_size = 512
cfg.batch_size = 96
cfg.img_backbone_opt = "resnet18"
cfg.cap_backbone_opt = "bigru"
cfg.dim = (384,128)
cfg.dist_fn_opt = "cosine"


In [4]:
from datasets.WIDERTriplet import *
from datasets.tokenizers import *
from datasets.text_test_datasets import *


class WIDERImageDataset(data.Dataset):
    def __init__(self,img_dir,transform=None,split='val',debug=False):
        super(WIDERImageDataset,self).__init__()
        self.img_dir = img_dir
        if split == 'val' or split == 'train':
            subfolders = os.listdir(img_dir)
            self.img_fns = []
            for subfolder in subfolders:
                for fn in os.listdir(os.path.join(img_dir,subfolder)):
                    self.img_fns.append(os.path.join(subfolder,fn))
        else:
            self.img_fns = os.listdir(img_dir)
        if debug:
            self.img_fns = self.img_fns[:500]
        #self.img_fns = [img for img in self.img_fns if img.endswith('.jpg')]
        self.transform=transform
    
    def get_all_keys(self):
        return self.img_fns
    
    def __len__(self):
        return len(self.img_fns)
    
    def _load_img(self,index):
        img_fn = self.img_fns[index]
        fn = os.path.join(self.img_dir,img_fn)
        image = Image.open(fn).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image,img_fn
    
    def __getitem__(self,index):
        image,image_fn = self._load_img(index)
        return image,image_fn
    
def build_test_image_loader(cfg):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    
    transform_test = transforms.Compose([
                                    transforms.Resize(dim),
                                    transforms.ToTensor(),
                                    normalize,
                            ])
    ds_image = WIDERImageDataset(img_dir=cfg.img_dir, transform=transform_test, debug=cfg.debug)
    dl_image = data.DataLoader(ds_image,
                               batch_size=cfg.batch_size,
                               shuffle=False,
                               num_workers=cfg.num_workers, 
                               pin_memory=True)
    
    

In [5]:
test_text_loader = build_test_text_loader(cfg)

NameError: name 'build_test_text_loader' is not defined

In [ ]:
captions = test_text_loader.dataset.captions
print(len(captions))
print(len(test_text_loader.dataset))
test_text_loader.dataset[0][0].size()
for data in test_text_loader:
    caps, nps, np2cap, image_fn = data
    import pdb; pdb.set_trace()

In [ ]:
index = 56


print(captions[index].lower())
print(sent_parse(captions[index]))
print(find_nps(captions[index]))